In [9]:
import pandas as pd
import seaborn as sns

%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## Base Tables

In [10]:
params = {
    'report_start_date': '2023-02-01',
    'report_end_date': '2023-02-28'
}

In [11]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs_v3_Feb23` AS (
    SELECT DISTINCT adobe_tracking_id
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Unsubscribes' 
    AND event_date <= @report_end_date
)

Query is running:   0%|          |

""


In [12]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted_v3_Feb23` AS ( -- everyone who have received emails in the month, and opened at least one email in the past
    SELECT DISTINCT adobe_tracking_id 
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Deliveries' 
    AND event_date BETWEEN @report_start_date and @report_end_date
)

Query is running:   0%|          |

""


In [13]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout_v3_Feb23` AS ( 
    SELECT  distinct holdout.adobe_tracking_id
    FROM
    (
        SELECT  distinct TrackingId AS adobe_tracking_id -- distinct Hold_Out_Type_Current --cohort, count(distinct TrackingId)
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = format_timestamp('%B%Y', DATETIME_TRUNC(@report_start_date, QUARTER)) -- get cohort name as month of quarter start + year
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        -- Exclude those who are assigned to Email Holdout but actually received emails 
    ) holdout
    LEFT JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries'
        AND event_date BETWEEN @report_start_date AND @report_end_date
        AND lower(campaign_name) NOT LIKE 'transactional%' -- Exclude transactional emails
    ) delivered -- Email Delivered ever
    ON holdout.adobe_tracking_id = delivered.adobe_tracking_id AND delivered.adobe_tracking_id is null
)

Query is running:   0%|          |

""


### Output Base Tables

In [14]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v3_delivered_4M_Feb23` AS

WITH 
email_engagers AS (
    SELECT aid
    FROM 
    (
        SELECT DISTINCT identity 
        FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
        WHERE eventName = 'Email Deliveries'
        -- 4 months before start of the holdout period, defined as start of quarter
        AND date(eventTimestamp) BETWEEN DATE_SUB(DATETIME_TRUNC(@report_start_date, QUARTER), INTERVAL 4 MONTH) AND @report_end_date
        AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails
    ) a
    
    INNER JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Braze_Id_Adobe_Id_Map` mapping
    ON mapping.bid = a.identity
)
, email_engagers_or_new_users AS (
    SELECT aid
    FROM email_engagers

    UNION ALL

    SELECT distinct adobe_tracking_id as aid
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
    WHERE registration_date >= DATE_SUB(@report_start_date, INTERVAL 4 MONTH)
)

SELECT  distinct a.adobe_tracking_id AS aid
       ,cohort
       ,user.account_type
       ,user.account_tenure
FROM email_engagers_or_new_users base

INNER JOIN
    (
        SELECT *, 'Email_Targeted' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted_v3_Feb23`
        UNION ALL
        SELECT *, 'Holdout' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout_v3_Feb23`
    ) a
ON a.adobe_tracking_id = base.aid
    
--add attribute: account_type 
INNER JOIN
(
	SELECT  adobe_tracking_id
	       ,account_type
	       ,account_tenure
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
	WHERE report_date = @report_end_date 
) user
ON a.adobe_tracking_id = user.adobe_tracking_id

-- for after 2021/july, email channel only, take out all abandon MAAs
INNER JOIN
(
    SELECT DISTINCT adobe_tracking_id
	FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES`
	WHERE report_date = @report_end_date
	AND date_of_last_view IS NOT NULL 
) abandon_maa
ON a.adobe_tracking_id = abandon_maa.adobe_tracking_id

-- exclude unsubscribed
LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Unsubs_v3_Feb23` unsub
ON a.adobe_tracking_id = unsub.adobe_tracking_id
WHERE unsub.adobe_tracking_id IS NULL

Query is running:   0%|          |

""


In [15]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Video_Viewing_v3_delivered_4M_Feb23` AS (
    SELECT 
        adobe_tracking_id
        , COUNT (DISTINCT CASE WHEN VIDEO.num_views_started = 1 THEN video.adobe_tracking_id ELSE NULL END) AS Distinct_Content_Starts
        , SUM (VIDEO.num_views_started ) AS Total_Content_Starts
        , SUM(VIDEO.num_seconds_played_no_ads)/3600 AS Viewing_Time
        , COUNT(DISTINCT session_id) AS Distinct_Viewing_Sessions 
        , COUNT(DISTINCT(CASE WHEN (num_seconds_played_no_ads > CASE WHEN lower(consumption_type) = 'virtual channel' THEN 299 ELSE 0 END)
                and (num_views_started>0) THEN CASE WHEN (lower(consumption_type) = "shortform") THEN "Shortform"
                                                    WHEN lower(franchise) != 'other' THEN franchise ELSE display_name
                                                    END
                end)) as Repertoire_Pavo_Method
                
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v3_delivered_4M_Feb23` a

    INNER JOIN  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO
        ON VIDEO.adobe_tracking_id = a.aid
        AND adobe_date between @report_start_date AND @report_end_date
    GROUP BY 1
)

Query is running:   0%|          |

""


## Metric Definitions

In [16]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v3_delivered_4M_Feb23` AS 

WITH
Save_Denom AS ( --'Lapsing_Users'
    SELECT distinct adobe_tracking_id, date_of_last_view -- no need for max (days_since_last_view), can be saved multiple times, dedup later
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
    WHERE report_date BETWEEN @report_start_date AND @report_end_date 
    AND days_since_last_view BETWEEN 15 AND 29 -- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.
)
, Save_Num as (
        SELECT distinct a.adobe_tracking_id      
        FROM Save_Denom a
        INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO 
            ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
            AND adobe_date BETWEEN @report_start_date AND @report_end_date
            AND VIDEO.adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 15 day) AND DATE_ADD(date_of_last_view, INTERVAL 29 day)
)
, Winback_Denom AS ( --'Lapsed_Users'
    SELECT distinct adobe_tracking_id, date_of_last_view
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
    WHERE report_date BETWEEN @report_start_date AND @report_end_date 
    AND days_since_last_view BETWEEN 30 AND 90-- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.
    )
-- REVIEW: compare this code to email dash (Gold table)
, Winback_Num AS (
    SELECT distinct a.adobe_tracking_id
    FROM Winback_Denom a
    INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`  VIDEO 
        ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
        AND adobe_date BETWEEN @report_start_date AND @report_end_date 
        AND VIDEO.adobe_date BETWEEN DATE_ADD(date_of_last_view, INTERVAL 30 day) 
            AND DATE_ADD(date_of_last_view, INTERVAL 90 DAY)
    )
, Upgrade_Denom AS (
-- upgrade metric 2.0: nonpaying to paying
    SELECT distinct adobe_tracking_id 
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
    WHERE paying_account_flag = 'NonPaying' 
    AND USER.report_date BETWEEN @report_start_date and @report_end_date
)
, Upgrade_Num AS (
-- find all users who have upgraded at least once in the month of
    SELECT  distinct adobe_tracking_id
    FROM
        (SELECT  
                report_date
             , adobe_tracking_id     
         FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
         WHERE entitlement_change_flag IN ('Upgrade: Free to Premium'
                                         , 'Upgrade: Free to Premium+'
                                         , 'Upgrade: Premium to Premium+') 
         AND paying_account_flag = 'Paying'                                            
            AND USER.report_date BETWEEN @report_start_date and @report_end_date
        ) 
)    
, Paid_Churn_Denom AS (
    SELECT distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
    WHERE paying_account_flag = 'Paying'
    AND auto_renew_flag = 'OFF'
    AND report_date BETWEEN @report_start_date and @report_end_date
)
, Paid_Churn_Num AS (
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  adobe_tracking_id
            ,report_date
            ,auto_renew_flag                                                                      AS auto_renew_flag_today
            ,LEAD(auto_renew_flag,1) OVER ( partition by adobe_tracking_id ORDER BY report_date ) AS auto_renew_flag_next_day
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE report_date BETWEEN @report_start_date AND @report_end_date
        ORDER BY 1, 2 
    )
    WHERE auto_renew_flag_today = 'OFF'
    AND auto_renew_flag_next_day = 'ON'
)
, New_Upgrade_Denom AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never`
    WHERE report_date BETWEEN @report_start_date AND @report_end_date
)
, New_Upgrade_Num AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`
    WHERE upgrade_row_number = 1
    AND report_date BETWEEN @report_start_date AND @report_end_date
)
, Paid_Winbacks_Denom AS (
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  adobe_tracking_id
            ,report_date
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE paying_account_flag = 'NonPaying'
        AND report_date BETWEEN @report_start_date AND @report_end_date 
    )
    WHERE adobe_tracking_id NOT IN ( SELECT distinct adobe_tracking_id FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_never` WHERE report_date BETWEEN @report_start_date AND @report_end_date)
)
, Paid_Winbacks_Num AS (
    select distinct adobe_tracking_id
    from `nbcu-ds-sandbox-a-001.SLi_sandbox.upgrade_date_rank`
    where upgrade_row_number > 1
    and report_date between @report_start_date and @report_end_date
)
-- this metric uses silver_churn, which is based on data in PAVO dash Churn Trend
, EOM_Paid_Churn_Denom AS (
  select adobe_tracking_id
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
  where base_date = @report_end_date
  and entitlement = 'Paid'
)
, EOM_Paid_Churn_Num AS (
  select adobe_tracking_id
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
  where base_date = @report_end_date
  and entitlement = 'Paid'
  and Churn_flag = 'Churn'
)
SELECT  @report_start_date                     AS Report_Month
       ,a.Account_Type
       ,a.Account_Tenure
       ,a.aid
       ,a.cohort
       ,video.adobe_tracking_id                AS viewers
       ,video.Viewing_Time
       ,video.Repertoire_Pavo_Method
       ,video.Distinct_Viewing_Sessions
       ,Winback_Denom.adobe_tracking_id        AS Winback_Denom
       ,Winback_Num.adobe_tracking_id          AS Winback_Num
       ,Save_Denom.adobe_tracking_id           AS Save_Denom
       ,Save_Num.adobe_tracking_id             AS Save_Num
       ,Upgrade_Denom.adobe_tracking_id        AS Upgrade_Denom
       ,Upgrade_Num.adobe_tracking_id          AS Upgrade_Num
       ,New_Upgrade_Denom.adobe_tracking_id    AS New_Upgrade_Denom
       ,New_Upgrade_Num.adobe_tracking_id      AS New_Upgrade_Num
       ,Paid_Winbacks_Denom.adobe_tracking_id  AS Paid_Winbacks_Denom
       ,Paid_Winbacks_Num.adobe_tracking_id    AS Paid_Winbacks_Num
       ,Paid_Churn_Denom.adobe_tracking_id     AS Paid_Churn_Denom
       ,Paid_Churn_Num.adobe_tracking_id       AS Paid_Churn_Num
       ,EOM_Paid_Churn_Denom.adobe_tracking_id AS EOM_Paid_Churn_Denom
       ,EOM_Paid_Churn_Num.adobe_tracking_id   AS EOM_Paid_Churn_Num
FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_v3_delivered_4M_Feb23` a
LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Video_Viewing_v3_delivered_4M_Feb23` video
    ON a.aid = video.adobe_tracking_id
LEFT JOIN Winback_Denom
    ON a.aid = Winback_Denom.adobe_tracking_id
LEFT JOIN Winback_Num
    ON Winback_Denom.adobe_tracking_id = Winback_Num.adobe_tracking_id
LEFT JOIN Save_Denom
    ON a.aid = Save_Denom.adobe_tracking_id
LEFT JOIN Save_Num
    ON Save_Denom.adobe_tracking_id = Save_Num.adobe_tracking_id
LEFT JOIN Upgrade_Denom
    ON a.aid = Upgrade_Denom.adobe_tracking_id
LEFT JOIN Upgrade_Num 
    ON Upgrade_Denom.adobe_tracking_id = Upgrade_Num.adobe_tracking_id
LEFT JOIN New_Upgrade_Denom
    on a.aid = New_Upgrade_Denom.adobe_tracking_id
LEFT JOIN New_Upgrade_Num
    on New_Upgrade_Denom.adobe_tracking_id = New_Upgrade_Num.adobe_tracking_id
LEFT JOIN Paid_Winbacks_Denom
    on a.aid = Paid_Winbacks_Denom.adobe_tracking_id
LEFT JOIN Paid_Winbacks_Num
    on Paid_Winbacks_Num.adobe_tracking_id = Paid_Winbacks_Denom.adobe_tracking_id
LEFT JOIN Paid_Churn_Denom
    on a.aid = Paid_Churn_Denom.adobe_tracking_id
LEFT JOIN Paid_Churn_Num
    on Paid_Churn_Denom.adobe_tracking_id = Paid_Churn_Num.adobe_tracking_id
LEFT JOIN EOM_Paid_Churn_Denom
    on a.aid = EOM_Paid_Churn_Denom.adobe_tracking_id
LEFT JOIN EOM_Paid_Churn_Num
    on EOM_Paid_Churn_Denom.adobe_tracking_id = EOM_Paid_Churn_Num.adobe_tracking_id

Query is running:   0%|          |

""


# Analysis

In [17]:
%%bigquery df_total --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
  SELECT 
    @report_start_date AS Report_Month
--    , Account_Type
    , count(distinct case when cohort = 'Email_Targeted' then aid end ) as Distinct_Cohort_Size_Targeted
    , count(distinct case when cohort = 'Holdout' then aid end) as Distinct_Cohort_Size_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then viewers end ) as Total_Returns_Targeted
    , count(distinct case when cohort = 'Holdout' then viewers end) as Total_Returns_Holdout

    , sum(case when cohort = 'Email_Targeted' then Viewing_Time end ) as Total_Usage_Targeted
    , sum(case when cohort = 'Holdout' then Viewing_Time end) as Total_Usage_Holdout

    , sum(case when cohort = 'Email_Targeted' then Repertoire_Pavo_Method end ) as Total_Repertoire_Targeted
    , sum(case when cohort = 'Holdout' then Repertoire_Pavo_Method end) as Total_Repertoire_Holdout

    , sum(case when cohort = 'Email_Targeted' then Distinct_Viewing_Sessions end ) as Total_Viewing_Sessions_Targeted
    , sum(case when cohort = 'Holdout' then Distinct_Viewing_Sessions end) as Total_Viewing_Sessions_Holdout


    , count(distinct case when cohort = 'Email_Targeted' then Winback_Denom end ) as Winback_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then Winback_Num end) as Winback_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then Winback_Denom end ) as Winback_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then Winback_Num end) as Winback_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then Save_Denom end ) as Save_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then Save_Num end) as Save_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then Save_Denom end ) as Save_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then Save_Num end) as Save_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then Upgrade_Denom end ) as Upgrades_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then Upgrade_Num end) as Upgrades_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then Upgrade_Denom end ) as Upgrades_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then Upgrade_Num end) as Upgrades_Num_Holdout


    , count(distinct case when cohort = 'Email_Targeted' then New_Upgrade_Denom end ) as Total_New_Upgrade_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then New_Upgrade_Num end) as Total_New_Upgrade_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then New_Upgrade_Denom end ) as Total_New_Upgrade_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then New_Upgrade_Num end) as Total_New_Upgrade_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then Paid_Winbacks_Denom end ) as Total_Paid_Winbacks_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then Paid_Winbacks_Num end) as Total_Paid_Winbacks_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then Paid_Winbacks_Denom end ) as Total_Paid_Winbacks_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then Paid_Winbacks_Num end) as Total_Paid_Winbacks_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then Paid_Churn_Denom end ) as Total_Paid_Churn_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then Paid_Churn_Num end) as Total_Paid_Churn_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then Paid_Churn_Denom end ) as Total_Paid_Churn_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then Paid_Churn_Num end) as Total_Paid_Churn_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Holdout
  FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v3_delivered_4M_Feb23`
  GROUP BY 1 --,2
), CTE_2 AS (
    SELECT Report_Month
--  , Account_Type 
  , Distinct_Cohort_Size_Targeted   as Email_Engagers
  , Distinct_Cohort_Size_Holdout   as Email_Holdout


  , safe_divide(Total_Returns_Targeted, Distinct_Cohort_Size_Targeted)  as Return_Rate_Engagers
  , safe_divide(Total_Returns_Holdout, Distinct_Cohort_Size_Holdout)  as Return_Rate_Holdout


  , safe_divide(Total_Usage_Targeted, Distinct_Cohort_Size_Targeted)  as Usage_Engagers
  , safe_divide(Total_Usage_Holdout, Distinct_Cohort_Size_Holdout)   as Usage_Holdout

  , safe_divide(Total_Repertoire_Targeted, Distinct_Cohort_Size_Targeted)  as Repertoire_Engagers
  , safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)   as Repertoire_Holdout


  , safe_divide(Total_Viewing_Sessions_Targeted, Distinct_Cohort_Size_Targeted)  as Sessions_Engagers
  , safe_divide(Total_Viewing_Sessions_Holdout, Distinct_Cohort_Size_Holdout)   as Sessions_Holdout

  , Winback_Denom_Targeted 
  , safe_divide(Winback_Num_Targeted, Winback_Denom_Targeted)  as Winback_Rate_Engagers
  , safe_divide(Winback_Num_Holdout, Winback_Denom_Holdout) as Winback_Rate_Holdout

  , Save_Denom_Targeted 
  , safe_divide(Save_Num_Targeted, Save_Denom_Targeted)  as Save_Rate_Engagers
  , safe_divide(Save_Num_Holdout, Save_Denom_Holdout) as Save_Rate_Holdout

  , Upgrades_Denom_Targeted
  , safe_divide(Upgrades_Num_Targeted, Upgrades_Denom_Targeted)  as Upgrade_Rate_Engagers
  , safe_divide(Upgrades_Num_Holdout, Upgrades_Denom_Holdout) as Upgrade_Rate_Holdout

  , Total_New_Upgrade_Denom_Targeted
  , safe_divide(Total_New_Upgrade_Num_Targeted, Total_New_Upgrade_Denom_Targeted)  as New_Upgrade_Rate_Engagers
  , safe_divide(Total_New_Upgrade_Num_Holdout,  Total_New_Upgrade_Denom_Holdout) as New_Upgrade_Rate_Holdout

  , Total_Paid_Winbacks_Denom_Targeted
  , safe_divide(Total_Paid_Winbacks_Num_Targeted, Total_Paid_Winbacks_Denom_Targeted)  as Paid_Winback_Rate_Engagers 
  , safe_divide(Total_Paid_Winbacks_Num_Holdout,  Total_Paid_Winbacks_Denom_Holdout) as Paid_Winback_Rate_Holdout 

  , Total_Paid_Churn_Denom_Targeted
  , safe_divide(Total_Paid_Churn_Num_Targeted, Total_Paid_Churn_Denom_Targeted) as Paid_Churn_Save_Rate_Engagers
  , safe_divide(Total_Paid_Churn_Num_Holdout, Total_Paid_Churn_Denom_Holdout) as Paid_Churn_Save_Rate_Holdout

  , EOM_Paid_Churn_Denom_Targeted
  , safe_divide(EOM_Paid_Churn_Num_Targeted, EOM_Paid_Churn_Denom_Targeted) as Paid_Churn_Rate_Engagers
  , safe_divide(EOM_Paid_Churn_Num_Holdout, EOM_Paid_Churn_Denom_Holdout) as Paid_Churn_Rate_Holdout

  from CTE_1
)

SELECT Report_Month
--, Account_Type
, Email_Engagers
, Email_Holdout


, Return_Rate_Engagers                                                   as Return_Rate_Engagers
, Return_Rate_Holdout                                                    as Return_Rate_Holdout
, Return_Rate_Engagers - Return_Rate_Holdout                             as Return_Rate_Lift_PTS
, safe_divide(Return_Rate_Engagers, Return_Rate_Holdout) *100                        as Return_Rate_Lift_Index
, (Return_Rate_Engagers - Return_Rate_Holdout) * Email_Engagers          as Returns_Incrementals

, Usage_Engagers                                                         as Usage_Engagers
, Usage_Holdout                                                          as Usage_Holdout
, Usage_Engagers - Usage_Holdout                                         as Usage_Lift_PTS
, safe_divide(Usage_Engagers, Usage_Holdout) *100                                    as Usage_Lift_Index
, (Usage_Engagers - Usage_Holdout) * Email_Engagers                      as Usage_Incrementals


, Repertoire_Engagers                                                    as Repertoire_Engagers
, Repertoire_Holdout                                                     as Repertoire_Holdout
, Repertoire_Engagers - Repertoire_Holdout                               as Repertoire_Lift_PTS
, safe_divide(Repertoire_Engagers, Repertoire_Holdout) *100                          as Repertoire_Lift_Index
, (Repertoire_Engagers - Repertoire_Holdout) * Email_Engagers as Repertoire_Incrementals


, Sessions_Engagers                                                      as Sessions_Engagers
, Sessions_Holdout                                                       as Sessions_Holdout
, Sessions_Engagers - Sessions_Holdout                                   as Sessions_Lift_PTS
, safe_divide(Sessions_Engagers, Sessions_Holdout) *100                              as Sessions_Lift_Index
, (Sessions_Engagers - Sessions_Holdout) * Email_Engagers as Sessions_Incrementals


, Winback_Rate_Engagers                                                   as Winback_Rate_Engagers
, Winback_Rate_Holdout                                                    as Winback_Rate_Holdout
, Winback_Rate_Engagers - Winback_Rate_Holdout                            as Winback_Rate_Lift_PTS
, safe_divide(Winback_Rate_Engagers, Winback_Rate_Holdout) *100                       as Winback_Rate_Lift_Index
, (Winback_Rate_Engagers - Winback_Rate_Holdout) * Winback_Denom_Targeted as Winback_Incrementals


, Save_Rate_Engagers                                                      as Save_Rate_Engagers
, Save_Rate_Holdout                                                       as Save_Rate_Holdout
, Save_Rate_Engagers - Save_Rate_Holdout                                  as Save_Rate_Lift_PTS
, safe_divide(Save_Rate_Engagers, Save_Rate_Holdout) *100                             as Save_Rate_Lift_Index
, (Save_Rate_Engagers - Save_Rate_Holdout) * Save_Denom_Targeted          as Save_Rate_Lift_Incrementals


, Upgrade_Rate_Engagers                                                    as Upgrade_Rate_Engagers
, Upgrade_Rate_Holdout                                                     as Upgrade_Rate_Holdout
, Upgrade_Rate_Engagers - Upgrade_Rate_Holdout                             as Upgrade_Rate_Lift_PTS
, safe_divide(Upgrade_Rate_Engagers, Upgrade_Rate_Holdout) *100                        as Upgrade_Rate_Lift_Index
, (Upgrade_Rate_Engagers - Upgrade_Rate_Holdout) * Upgrades_Denom_Targeted as Upgrade_Incrementals


, New_Upgrade_Rate_Engagers                                                as New_Upgrade_Rate_Engagers
, New_Upgrade_Rate_Holdout                                                 as New_Upgrade_Rate_Holdout
, New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout                     as New_Upgrade_Rate_Lift_PTS
, safe_divide(New_Upgrade_Rate_Engagers, New_Upgrade_Rate_Holdout) *100                as New_Upgrade_Rate_Lift_Index
, (New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout) * Total_New_Upgrade_Denom_Targeted as New_Upgrade_Incrementals

, Paid_Winback_Rate_Engagers                                               as Paid_Winback_Rate_Engagers
, Paid_Winback_Rate_Holdout                                                as Paid_Winback_Rate_Holdout
, Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout                   as Paid_Winback_Rate_Lift_PTS
, safe_divide(Paid_Winback_Rate_Engagers, Paid_Winback_Rate_Holdout) *100              as Paid_Winback_Rate_Lift_Index
, (Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout) * Total_Paid_Winbacks_Denom_Targeted as Paid_Winback_Rate_Lift_Incrementals

, Paid_Churn_Save_Rate_Engagers                                             as Paid_Churn_Save_Rate_Engagers
, Paid_Churn_Save_Rate_Holdout                                              as Paid_Churn_Save_Rate_Holdout
, Paid_Churn_Save_Rate_Engagers - Paid_Churn_Save_Rate_Holdout              as Paid_Churn_Save_Rate_Lift_PTS
, safe_divide(Paid_Churn_Save_Rate_Engagers, Paid_Churn_Save_Rate_Holdout) *100         as Paid_Churn_Save_Rate_Lift_Index
, (Paid_Churn_Save_Rate_Engagers - Paid_Churn_Save_Rate_Holdout) * Total_Paid_Churn_Denom_Targeted as Paid_Churn_Save_Rate_Incrementals

, Paid_Churn_Rate_Engagers                                                  as EOM_Paid_Churn_Rate_Engagers
, Paid_Churn_Rate_Holdout                                                   as EOM_Paid_Churn_Rate_Holdout
, Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                        as EOM_Paid_Churn_Rate_Lift_PTS
, safe_divide(Paid_Churn_Rate_Engagers, Paid_Churn_Rate_Holdout) *100                   as EOM_Paid_Churn_Rate_Lift_Index
, (Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted as EOM_Paid_Churn_Rate_Incrementals


FROM CTE_2

Query is running:   0%|          |

Downloading:   0%|          |

In [18]:
df_total

,Report_Month,Email_Engagers,Email_Holdout,Return_Rate_Engagers,Return_Rate_Holdout,Return_Rate_Lift_PTS,Return_Rate_Lift_Index,Returns_Incrementals,Usage_Engagers,Usage_Holdout,...,Paid_Churn_Save_Rate_Engagers,Paid_Churn_Save_Rate_Holdout,Paid_Churn_Save_Rate_Lift_PTS,Paid_Churn_Save_Rate_Lift_Index,Paid_Churn_Save_Rate_Incrementals,EOM_Paid_Churn_Rate_Engagers,EOM_Paid_Churn_Rate_Holdout,EOM_Paid_Churn_Rate_Lift_PTS,EOM_Paid_Churn_Rate_Lift_Index,EOM_Paid_Churn_Rate_Incrementals
0,2023-02-01,40590132,1816416,0.465472,0.357669,0.107804,130.140632,4.375762e+06,8.52478,6.329523,...,0.029346,0.026679,0.002667,109.995244,1185.305406,NaN,NaN,NaN,NaN,NaN


In [19]:
%%bigquery df_acc --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
  SELECT 
    @report_start_date AS Report_Month
    , Account_Type
    , count(distinct case when cohort = 'Email_Targeted' then aid end ) as Distinct_Cohort_Size_Targeted
    , count(distinct case when cohort = 'Holdout' then aid end) as Distinct_Cohort_Size_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then viewers end ) as Total_Returns_Targeted
    , count(distinct case when cohort = 'Holdout' then viewers end) as Total_Returns_Holdout

    , sum(case when cohort = 'Email_Targeted' then Viewing_Time end ) as Total_Usage_Targeted
    , sum(case when cohort = 'Holdout' then Viewing_Time end) as Total_Usage_Holdout

    , sum(case when cohort = 'Email_Targeted' then Repertoire_Pavo_Method end ) as Total_Repertoire_Targeted
    , sum(case when cohort = 'Holdout' then Repertoire_Pavo_Method end) as Total_Repertoire_Holdout

    , sum(case when cohort = 'Email_Targeted' then Distinct_Viewing_Sessions end ) as Total_Viewing_Sessions_Targeted
    , sum(case when cohort = 'Holdout' then Distinct_Viewing_Sessions end) as Total_Viewing_Sessions_Holdout


    , count(distinct case when cohort = 'Email_Targeted' then Winback_Denom end ) as Winback_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then Winback_Num end) as Winback_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then Winback_Denom end ) as Winback_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then Winback_Num end) as Winback_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then Save_Denom end ) as Save_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then Save_Num end) as Save_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then Save_Denom end ) as Save_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then Save_Num end) as Save_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then Upgrade_Denom end ) as Upgrades_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then Upgrade_Num end) as Upgrades_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then Upgrade_Denom end ) as Upgrades_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then Upgrade_Num end) as Upgrades_Num_Holdout


    , count(distinct case when cohort = 'Email_Targeted' then New_Upgrade_Denom end ) as Total_New_Upgrade_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then New_Upgrade_Num end) as Total_New_Upgrade_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then New_Upgrade_Denom end ) as Total_New_Upgrade_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then New_Upgrade_Num end) as Total_New_Upgrade_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then Paid_Winbacks_Denom end ) as Total_Paid_Winbacks_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then Paid_Winbacks_Num end) as Total_Paid_Winbacks_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then Paid_Winbacks_Denom end ) as Total_Paid_Winbacks_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then Paid_Winbacks_Num end) as Total_Paid_Winbacks_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then Paid_Churn_Denom end ) as Total_Paid_Churn_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then Paid_Churn_Num end) as Total_Paid_Churn_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then Paid_Churn_Denom end ) as Total_Paid_Churn_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then Paid_Churn_Num end) as Total_Paid_Churn_Num_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Holdout
  FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v3_delivered_4M_Feb23`
  GROUP BY 1 ,2
), CTE_2 AS (
    SELECT Report_Month
  , Account_Type 
  , Distinct_Cohort_Size_Targeted   as Email_Engagers
  , Distinct_Cohort_Size_Holdout   as Email_Holdout


  , safe_divide(Total_Returns_Targeted, Distinct_Cohort_Size_Targeted)  as Return_Rate_Engagers
  , safe_divide(Total_Returns_Holdout, Distinct_Cohort_Size_Holdout)  as Return_Rate_Holdout


  , safe_divide(Total_Usage_Targeted, Distinct_Cohort_Size_Targeted)  as Usage_Engagers
  , safe_divide(Total_Usage_Holdout, Distinct_Cohort_Size_Holdout)   as Usage_Holdout

  , safe_divide(Total_Repertoire_Targeted, Distinct_Cohort_Size_Targeted)  as Repertoire_Engagers
  , safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)   as Repertoire_Holdout


  , safe_divide(Total_Viewing_Sessions_Targeted, Distinct_Cohort_Size_Targeted)  as Sessions_Engagers
  , safe_divide(Total_Viewing_Sessions_Holdout, Distinct_Cohort_Size_Holdout)   as Sessions_Holdout

  , Winback_Denom_Targeted 
  , safe_divide(Winback_Num_Targeted, Winback_Denom_Targeted)  as Winback_Rate_Engagers
  , safe_divide(Winback_Num_Holdout, Winback_Denom_Holdout) as Winback_Rate_Holdout

  , Save_Denom_Targeted 
  , safe_divide(Save_Num_Targeted, Save_Denom_Targeted)  as Save_Rate_Engagers
  , safe_divide(Save_Num_Holdout, Save_Denom_Holdout) as Save_Rate_Holdout

  , Upgrades_Denom_Targeted
  , safe_divide(Upgrades_Num_Targeted, Upgrades_Denom_Targeted)  as Upgrade_Rate_Engagers
  , safe_divide(Upgrades_Num_Holdout, Upgrades_Denom_Holdout) as Upgrade_Rate_Holdout

  , Total_New_Upgrade_Denom_Targeted
  , safe_divide(Total_New_Upgrade_Num_Targeted, Total_New_Upgrade_Denom_Targeted)  as New_Upgrade_Rate_Engagers
  , safe_divide(Total_New_Upgrade_Num_Holdout,  Total_New_Upgrade_Denom_Holdout) as New_Upgrade_Rate_Holdout

  , Total_Paid_Winbacks_Denom_Targeted
  , safe_divide(Total_Paid_Winbacks_Num_Targeted, Total_Paid_Winbacks_Denom_Targeted)  as Paid_Winback_Rate_Engagers 
  , safe_divide(Total_Paid_Winbacks_Num_Holdout,  Total_Paid_Winbacks_Denom_Holdout) as Paid_Winback_Rate_Holdout 

  , Total_Paid_Churn_Denom_Targeted
  , safe_divide(Total_Paid_Churn_Num_Targeted, Total_Paid_Churn_Denom_Targeted) as Paid_Churn_Save_Rate_Engagers
  , safe_divide(Total_Paid_Churn_Num_Holdout, Total_Paid_Churn_Denom_Holdout) as Paid_Churn_Save_Rate_Holdout

  , EOM_Paid_Churn_Denom_Targeted
  , safe_divide(EOM_Paid_Churn_Num_Targeted, EOM_Paid_Churn_Denom_Targeted) as Paid_Churn_Rate_Engagers
  , safe_divide(EOM_Paid_Churn_Num_Holdout, EOM_Paid_Churn_Denom_Holdout) as Paid_Churn_Rate_Holdout

  from CTE_1
)

SELECT Report_Month
, Account_Type
, Email_Engagers
, Email_Holdout


, Return_Rate_Engagers                                                   as Return_Rate_Engagers
, Return_Rate_Holdout                                                    as Return_Rate_Holdout
, Return_Rate_Engagers - Return_Rate_Holdout                             as Return_Rate_Lift_PTS
, safe_divide(Return_Rate_Engagers, Return_Rate_Holdout) *100                        as Return_Rate_Lift_Index
, (Return_Rate_Engagers - Return_Rate_Holdout) * Email_Engagers          as Returns_Incrementals

, Usage_Engagers                                                         as Usage_Engagers
, Usage_Holdout                                                          as Usage_Holdout
, Usage_Engagers - Usage_Holdout                                         as Usage_Lift_PTS
, safe_divide(Usage_Engagers, Usage_Holdout) *100                                    as Usage_Lift_Index
, (Usage_Engagers - Usage_Holdout) * Email_Engagers                      as Usage_Incrementals


, Repertoire_Engagers                                                    as Repertoire_Engagers
, Repertoire_Holdout                                                     as Repertoire_Holdout
, Repertoire_Engagers - Repertoire_Holdout                               as Repertoire_Lift_PTS
, safe_divide(Repertoire_Engagers, Repertoire_Holdout) *100                          as Repertoire_Lift_Index
, (Repertoire_Engagers - Repertoire_Holdout) * Email_Engagers as Repertoire_Incrementals


, Sessions_Engagers                                                      as Sessions_Engagers
, Sessions_Holdout                                                       as Sessions_Holdout
, Sessions_Engagers - Sessions_Holdout                                   as Sessions_Lift_PTS
, safe_divide(Sessions_Engagers, Sessions_Holdout) *100                              as Sessions_Lift_Index
, (Sessions_Engagers - Sessions_Holdout) * Email_Engagers as Sessions_Incrementals


, Winback_Rate_Engagers                                                   as Winback_Rate_Engagers
, Winback_Rate_Holdout                                                    as Winback_Rate_Holdout
, Winback_Rate_Engagers - Winback_Rate_Holdout                            as Winback_Rate_Lift_PTS
, safe_divide(Winback_Rate_Engagers, Winback_Rate_Holdout) *100                       as Winback_Rate_Lift_Index
, (Winback_Rate_Engagers - Winback_Rate_Holdout) * Winback_Denom_Targeted as Winback_Incrementals


, Save_Rate_Engagers                                                      as Save_Rate_Engagers
, Save_Rate_Holdout                                                       as Save_Rate_Holdout
, Save_Rate_Engagers - Save_Rate_Holdout                                  as Save_Rate_Lift_PTS
, safe_divide(Save_Rate_Engagers, Save_Rate_Holdout) *100                             as Save_Rate_Lift_Index
, (Save_Rate_Engagers - Save_Rate_Holdout) * Save_Denom_Targeted          as Save_Rate_Lift_Incrementals


, Upgrade_Rate_Engagers                                                    as Upgrade_Rate_Engagers
, Upgrade_Rate_Holdout                                                     as Upgrade_Rate_Holdout
, Upgrade_Rate_Engagers - Upgrade_Rate_Holdout                             as Upgrade_Rate_Lift_PTS
, safe_divide(Upgrade_Rate_Engagers, Upgrade_Rate_Holdout) *100                        as Upgrade_Rate_Lift_Index
, (Upgrade_Rate_Engagers - Upgrade_Rate_Holdout) * Upgrades_Denom_Targeted as Upgrade_Incrementals


, New_Upgrade_Rate_Engagers                                                as New_Upgrade_Rate_Engagers
, New_Upgrade_Rate_Holdout                                                 as New_Upgrade_Rate_Holdout
, New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout                     as New_Upgrade_Rate_Lift_PTS
, safe_divide(New_Upgrade_Rate_Engagers, New_Upgrade_Rate_Holdout) *100                as New_Upgrade_Rate_Lift_Index
, (New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout) * Total_New_Upgrade_Denom_Targeted as New_Upgrade_Incrementals

, Paid_Winback_Rate_Engagers                                               as Paid_Winback_Rate_Engagers
, Paid_Winback_Rate_Holdout                                                as Paid_Winback_Rate_Holdout
, Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout                   as Paid_Winback_Rate_Lift_PTS
, safe_divide(Paid_Winback_Rate_Engagers, Paid_Winback_Rate_Holdout) *100              as Paid_Winback_Rate_Lift_Index
, (Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout) * Total_Paid_Winbacks_Denom_Targeted as Paid_Winback_Rate_Lift_Incrementals

, Paid_Churn_Save_Rate_Engagers                                             as Paid_Churn_Save_Rate_Engagers
, Paid_Churn_Save_Rate_Holdout                                              as Paid_Churn_Save_Rate_Holdout
, Paid_Churn_Save_Rate_Engagers - Paid_Churn_Save_Rate_Holdout              as Paid_Churn_Save_Rate_Lift_PTS
, safe_divide(Paid_Churn_Save_Rate_Engagers, Paid_Churn_Save_Rate_Holdout) *100         as Paid_Churn_Save_Rate_Lift_Index
, (Paid_Churn_Save_Rate_Engagers - Paid_Churn_Save_Rate_Holdout) * Total_Paid_Churn_Denom_Targeted as Paid_Churn_Save_Rate_Incrementals

, Paid_Churn_Rate_Engagers                                                  as EOM_Paid_Churn_Rate_Engagers
, Paid_Churn_Rate_Holdout                                                   as EOM_Paid_Churn_Rate_Holdout
, Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                        as EOM_Paid_Churn_Rate_Lift_PTS
, safe_divide(Paid_Churn_Rate_Engagers, Paid_Churn_Rate_Holdout) *100                   as EOM_Paid_Churn_Rate_Lift_Index
, (Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted as EOM_Paid_Churn_Rate_Incrementals


FROM CTE_2

Query is running:   0%|          |

Downloading:   0%|          |

In [20]:
df_acc = df_acc.sort_values(by='Account_Type')
df_acc

,Report_Month,Account_Type,Email_Engagers,Email_Holdout,Return_Rate_Engagers,Return_Rate_Holdout,Return_Rate_Lift_PTS,Return_Rate_Lift_Index,Returns_Incrementals,Usage_Engagers,...,Paid_Churn_Save_Rate_Engagers,Paid_Churn_Save_Rate_Holdout,Paid_Churn_Save_Rate_Lift_PTS,Paid_Churn_Save_Rate_Lift_Index,Paid_Churn_Save_Rate_Incrementals,EOM_Paid_Churn_Rate_Engagers,EOM_Paid_Churn_Rate_Holdout,EOM_Paid_Churn_Rate_Lift_PTS,EOM_Paid_Churn_Rate_Lift_Index,EOM_Paid_Churn_Rate_Incrementals
0,2023-02-01,Bundled Premium,8273537,298030,0.578135,0.508979,0.069156,113.587273,572167.726917,8.475222,...,0.006388,0.006623,-0.000235,96.453744,-1.066225,NaN,NaN,NaN,NaN,NaN
2,2023-02-01,Free,18712274,1032622,0.159476,0.112382,0.047094,141.905507,881239.499418,0.459454,...,0.000608,0.000501,0.000107,121.379288,26.772705,NaN,NaN,NaN,NaN,NaN
1,2023-02-01,Paying SVOD,13604321,485764,0.817843,0.786258,0.031584,104.017039,429683.049300,19.648498,...,0.067763,0.064711,0.003052,104.716067,579.307161,NaN,NaN,NaN,NaN,NaN


In [21]:
df_total.to_excel('email_kpi_tot_v3_delivered_4M_Feb23.xlsx', index=False)
df_acc.to_excel('email_kpi_acc_v3_delivered_4M_Feb23.xlsx', index=False)

### Churn Breakdown

In [22]:
%%bigquery df_churn --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
  SELECT 
    @report_start_date AS Report_Month
    , Account_Type
    , account_tenure
    , count(distinct case when cohort = 'Email_Targeted' then aid end ) as Distinct_Cohort_Size_Targeted
    , count(distinct case when cohort = 'Holdout' then aid end) as Distinct_Cohort_Size_Holdout

    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Targeted
    , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Targeted
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Denom end ) as EOM_Paid_Churn_Denom_Holdout
    , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Num end) as EOM_Paid_Churn_Num_Holdout
  FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v3_delivered_4M_Feb23`
  GROUP BY 1 ,2 ,3
), CTE_2 AS (
    SELECT Report_Month
  , Account_Type
  , account_tenure
  , Distinct_Cohort_Size_Targeted   as Email_Engagers
  , Distinct_Cohort_Size_Holdout   as Email_Holdout

  , EOM_Paid_Churn_Denom_Targeted
  , safe_divide(EOM_Paid_Churn_Num_Targeted, EOM_Paid_Churn_Denom_Targeted) as Paid_Churn_Rate_Engagers
  , safe_divide(EOM_Paid_Churn_Num_Holdout, EOM_Paid_Churn_Denom_Holdout) as Paid_Churn_Rate_Holdout

  from CTE_1
)

SELECT Report_Month
, Account_Type
, account_tenure
, Email_Engagers
, Email_Holdout

, Paid_Churn_Rate_Engagers                                                  as EOM_Paid_Churn_Rate_Engagers
, Paid_Churn_Rate_Holdout                                                   as EOM_Paid_Churn_Rate_Holdout
, Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                        as EOM_Paid_Churn_Rate_Lift_PTS
, safe_divide(Paid_Churn_Rate_Engagers, Paid_Churn_Rate_Holdout) *100                   as EOM_Paid_Churn_Rate_Lift_Index
, (Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted as EOM_Paid_Churn_Rate_Incrementals

FROM CTE_2
order by 1,2,3

Query is running:   0%|          |

Downloading:   0%|          |

In [23]:
df_churn.to_excel('email_kpi_v3_delivered_4M_Feb23_churn.xlsx')